# Simulating fast ion slowing-down process

<img src='../../figures/slowingdown.png' alt='thumbnail' width='200'/>

This example shows how to run a fast-ion slowing-down simulation which is the bread-and-butter for ASCOT5.

We begin this example by generating some general input. For slowing-down simulations we need at least bfield and plasma inputs to contain real data:

In [ ]:
import numpy as np
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)

# Use analytical field since it is fast to interpolate
a5.data.create_input("bfield analytical iter circular", desc="ANALYTICAL")

# DT-plasma
nrho   = 101
rho    = np.linspace(0, 2, nrho).T
edens  = 2e21 * np.ones((nrho, 1))
etemp  = 1e4  * np.ones((nrho, 1))
idens  = 1e21 * np.ones((nrho, 2))
itemp  = 1e4  * np.ones((nrho, 1))

edens[rho>1, :] = 1
idens[rho>1, :] = 1

pls = {
    "nrho" : nrho, "nion" : 2, "rho" : rho,
    "anum" : np.array([2, 3]), "znum" : np.array([1, 1]),
    "mass" : np.array([2.014, 3.016]), "charge" : np.array([1, 1]),
    "edensity" : edens, "etemperature" : etemp,
    "idensity" : idens, "itemperature" : itemp}
a5.data.create_input("plasma_1D", **pls, desc="FLATDT")

# These inputs are not relevant for this tutorial
a5.data.create_input("wall rectangular")
a5.data.create_input("E_TC")
a5.data.create_input("N0_1D")
a5.data.create_input("Boozer")
a5.data.create_input("MHD_STAT")
a5.data.create_input("asigma_loc")
print("Inputs created")

Next we create alpha source distribution using AFSI, but one could use BBNBI5 to generate beam ions or some other tool.

In [ ]:
# Grid spanning the whole plasma
rmin =  4.2; rmax = 8.2; nr = 50
zmin = -2.0; zmax = 2.0; nz = 50

# Run AFSI which creates a fusion alpha distribution
a5.afsi.thermal(
    1,
    rmin, rmax, nr, zmin, zmax, nz,
    minphi=0, maxphi=2*np.pi, nphi=1,
    nmc=100, mult=1.0,
    minppara=-1.3e-19, maxppara=1.3e-19, nppara=80,
    minpperp=0, maxpperp=1.3e-19, npperp=40)

# Convert to E-xi as then we can make dist more compact in momentum space
alphadist = a5.data.active.getdist("prod1", exi=True, ekin_edges=20, pitch_edges=10)
alphadist.integrate(time=np.s_[:], charge=np.s_[:]) # Distribution must be 5D


# Plot the distribution
#rzdist  = alphadist.integrate(copy=True, phi=np.s_[:], ppar=np.s_[:], pperp=np.s_[:])
rzdist  = alphadist.integrate(copy=True, phi=np.s_[:], ekin=np.s_[:], pitch=np.s_[:])
exidist = alphadist.integrate(copy=True, phi=np.s_[:], r=np.s_[:], z=np.s_[:])

fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

rzdist.plot(axes=ax1)
exidist.plot(axes=ax2)

plt.show()

Create markers by initializing marker distribution from a flat radial profile, and assigning weights from the alpha birth distribution.
Here we only initialize markers at the core to avoid losses.
AFSI and marker generation are covered in separate tutorials.

In [ ]:
# Create marker distribution. Here we just initialize markers evenly in rho for convenience
rho  = np.linspace(0, 0.5, 2)
prob = 0.5 * np.ones((2,))
a5.input_init(bfield=True)
markerdist = a5.markergen.rhoto5d(
    rho, prob, alphadist.abscissa_edges("r"),
    alphadist.abscissa_edges("phi"),  alphadist.abscissa_edges("z"),
    alphadist.abscissa_edges("ekin"), alphadist.abscissa_edges("pitch"))
a5.input_free()

# Create markers and take weights from the alpha distribution
mrk, mrkdist, prtdist = a5.markergen.generate(
    markerdist, alphadist, 4.002*unyt.amu, 2.0*unyt.e, 4, 2, 10**3, minweight=1e-10)
a5.data.create_input("gc", **mrk, desc="ALPHAS")

# Plot the marker and particle distributions
rzmrk  = mrkdist.integrate(copy=True, phi=np.s_[:], ekin=np.s_[:], pitch=np.s_[:])
eximrk = mrkdist.integrate(copy=True, phi=np.s_[:], r=np.s_[:], z=np.s_[:])
rzprt  = prtdist.integrate(copy=True, phi=np.s_[:], ekin=np.s_[:], pitch=np.s_[:])
exiprt = prtdist.integrate(copy=True, phi=np.s_[:], r=np.s_[:], z=np.s_[:])

fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)

rzmrk.plot(axes=ax1)
eximrk.plot(axes=ax2)
rzprt.plot(axes=ax3)
exiprt.plot(axes=ax4)

plt.show()


Now only options are missing.
If the guiding-center approximation is valid, then using adaptive step is strongly recommended to accelerate the simulation.
For other options, it is important to include energy limit as an end condition and activate orbit-following and Coulomb collisions in physics.
Diagnostics can be whatever you like, though usually it makes little sense to collect orbit data.
Here we collect 5D distribution which can be used to produce *the slowing-down distribution*.

In [ ]:
from a5py.ascot5io.options import Opt

opt = Opt.get_default()
opt.update({
    # Simulation mode
    "SIM_MODE":2, "ENABLE_ADAPTIVE":1,
    # Setting max mileage above slowing-down time is a good safeguard to ensure
    # simulation finishes even with faulty inputs. Same with the CPU time limit.
    "ENDCOND_SIMTIMELIM":1, "ENDCOND_MAX_MILEAGE":0.5,
    "ENDCOND_CPUTIMELIM":1, "ENDCOND_MAX_CPUTIME":1.0e1,
    # The energy limit which separates a fast ion from thermal bulk is not well defined,
    # but we usually use Emin = 2 x Tion as the limit. Setting also a fixed minimum energy
    # is advised sine plasma temperature at the pedestal can be low.
    "ENDCOND_ENERGYLIM":1, "ENDCOND_MIN_ENERGY":2.0e3, "ENDCOND_MIN_THERMAL":2.0,
    # Physics
    "ENABLE_ORBIT_FOLLOWING":1, "ENABLE_COULOMB_COLLISIONS":1,
    # Distribution output
    "ENABLE_DIST_5D":1,
    "DIST_MIN_R":4.3,        "DIST_MAX_R":8.3,       "DIST_NBIN_R":50,
    "DIST_MIN_PHI":0,        "DIST_MAX_PHI":360,     "DIST_NBIN_PHI":1,
    "DIST_MIN_Z":-2.0,       "DIST_MAX_Z":2.0,       "DIST_NBIN_Z":50,
    "DIST_MIN_PPA":-1.3e-19, "DIST_MAX_PPA":1.3e-19, "DIST_NBIN_PPA":100,
    "DIST_MIN_PPE":0,        "DIST_MAX_PPE":1.3e-19, "DIST_NBIN_PPE":50,
    "DIST_MIN_TIME":0,       "DIST_MAX_TIME":1.0,    "DIST_NBIN_TIME":1,
    "DIST_MIN_CHARGE":1,     "DIST_MAX_CHARGE":3,    "DIST_NBIN_CHARGE":1,
})
a5.data.create_input("opt", **opt, desc="SLOWINGDOWN")

Now we are ready to simulate.
Usually slowing-down simulations are expensive (especially with 3D magnetic field) but this one runs in a PC in a few minutes or so.

In [ ]:
import subprocess
subprocess.run(["./../../build/ascot5_main", "--d=\"SDALPHA\""])
print("Simulation completed")

Now the first thing to check in the results is that the markers actually slowed-down.
Therefore we look at the end conditions, final energy, and mileage which gives us the slowing-down time.

In [ ]:
a5 = Ascot("ascot.h5") # Re-read the data
print(a5.data.active.getstate_markersummary())
a5.data.active.plotstate_histogram("end ekin")
a5.data.active.plotstate_histogram("end mileage")

Looks reasonable!
Now let us look at the slowing-down distribution.

In [ ]:
d = a5.data.active.getdist("5d", exi=True, ekin_edges=np.linspace(0,5e6,40), pitch_edges=2)
d.integrate(time=np.s_[:], charge=np.s_[:], phi=np.s_[:], r=np.s_[:], z=np.s_[:], pitch=np.s_[:])
d.plot()

One can note that there might be markers above the initial energy, i.e. some markers have actually gained energy!
However, this "up-scattering" is fine because the collision operator contains a stochastic component and the main thing is that the whole population cools on average.
And, as we checked earlier, there were no energetic markers left at the end of the simulation so all markers either slowed-down or were lost.

Finally, it is a good idea to check the energy-pitch distribution as well.

In [ ]:
d = a5.data.active.getdist("5d", exi=True, ekin_edges=20, pitch_edges=10)
d.integrate(time=np.s_[:], charge=np.s_[:], phi=np.s_[:], r=np.s_[:], z=np.s_[:])
d.plot()

Here we want to verify that marker pitch has not accumulated in some point because the physics governs that the pitch scattering should make the pitch distribution close to even in the end.
Initially there is little pitch scattering, since fast particles are usually born above the critical energy,
$$v_\mathrm{crit} = v_\mathrm{th} \left(\frac{3 \sqrt{\pi}}{4}\frac{m_e}{m_\mathrm{ion}}\right)^{1/3}.$$

Above the critical energy we should see little pitch scattering as the fast ions mostly collide with electrons, but this changes below the critical energy as now the ion-ion collision frequency is higher than the electron-ion collision frequency.

This concludes the tutorial.
Here's a complementary fast-ion slowing-down density to brighten your day:

In [ ]:
d = a5.data.active.getdist("5d")
d.integrate(time=np.s_[:], charge=np.s_[:], phi=np.s_[:], ppar=np.s_[:], pperp=np.s_[:])
d.plot()